# Neuron Network Model (CPU)

In [12]:
import tensorflow as tf
import numpy as np
import pickle
import os
import matplotlib as plt
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import matplotlib.pyplot as plt
import time
import datetime
import scipy
from scipy.sparse import coo_matrix
# sess = tf.InteractiveSession()

**Confusion Matrix Plot**

In [2]:
def plot_confusion_matrix(cm, classes, normalize=False):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cmap = plt.cm.Blues
    title = "Confusion Matrix"
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=3)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

---
## 0 .Data Preparation
---

In [3]:
%%time
with open("../../Data/Propensity_Modeling/Neuron_Net/data.p", "rb" ) as f:
    data_features = pickle.load(f)    
with open( "../../Data/Feature_Engineering/df_label.p", "rb" ) as f:
    data_label = pickle.load(f)

CPU times: user 479 ms, sys: 2.57 s, total: 3.05 s
Wall time: 18 s


In [4]:
%%time
print('shape of data_features', data_features.shape)
print('shape of data_label', data_label.shape)
print('---------------------------------------')
X = data_features
Y = data_label
Y = np.asarray(Y)

bar_70 = np.floor(X.shape[0]*0.7).astype(int)
bar_70=bar_70+1
bar_80 = np.floor(X.shape[0]*0.8).astype(int)
bar_80=bar_80+1

X_train=X[:bar_70]
# X_train.reshape([X_train.shape[0],X_train.shape[1]])
# X_train = tf.reshape(X_train,[X_train.shape[0],X_train.shape[1]])
# X_train=tf.cast(X_train, tf.float32)
# X_train=X_train.eval()
print('X_train done.')

Y_train=Y[:bar_70]
Y_train=Y_train.reshape([Y_train.shape[0],1])
# Y_train = tf.reshape(Y_train,[Y_train.shape[0],1])
# Y_train=tf.cast(Y_train, tf.float32)
# Y_train=Y_train.eval()
print('Y_train done.')

X_validation=X[bar_70:bar_80]
# X_validataion.reshape([X_validataion.shape[0],X_validataion.shape[1]])
# X_validataion = tf.reshape(X_validataion,[X_validataion.shape[0],X_validataion.shape[1]])
# X_validataion=tf.cast(X_validataion, tf.float32)
# X_validataion=X_validataion.eval()
print('X_validation done.')

Y_validation=Y[bar_70:bar_80]
Y_validation=Y_validation.reshape([Y_validation.shape[0],1])
# Y_validation = tf.reshape(Y_validation,[Y_validation.shape[0],1])
# Y_validation=tf.cast(Y_validation, tf.float32)
# Y_validation=Y_validation.eval()
print('Y_validation done.')

X_test=X[bar_80:]
# X_test.reshape([X_test.shape[0],X_test.shape[1]])
# X_test = tf.reshape(X_test,[X_test.shape[0],X_test.shape[1]])
# X_test=tf.cast(X_test, tf.float32)
# X_test=X_test.eval()
print('X_test done.')

Y_test=Y[bar_80:]
Y_test=Y_test.reshape([Y_test.shape[0],1])
# Y_test = tf.reshape(Y_test,[Y_test.shape[0],1])
# Y_test=tf.cast(Y_test, tf.float32)
# Y_test=Y_test.eval()
print('Y_test done.')

print('---------------------------------------')
print('shape of X_train',X_train.shape)
print('shape of Y_train',Y_train.shape)
print('shape of X_validataion',X_validation.shape)
print('shape of Y_validation',Y_validation.shape)
print('shape of X_test',X_test.shape)
print('shape of Y_test',Y_test.shape)

shape of data_features (220047, 2351)
shape of data_label (220047,)
---------------------------------------
X_train done.
Y_train done.
X_validation done.
Y_validation done.
X_test done.
Y_test done.
---------------------------------------
shape of X_train (154033, 2351)
shape of Y_train (154033, 1)
shape of X_validataion (22005, 2351)
shape of Y_validation (22005, 1)
shape of X_test (44009, 2351)
shape of Y_test (44009, 1)
CPU times: user 2 ms, sys: 1e+03 µs, total: 3 ms
Wall time: 2.79 ms


In [5]:
(nrTrainSamples, dimX) = X_train.shape
(nrValidationSamples, dimY)  = Y_validation.shape

print ("Finished: data loaded. Stats below: ")
print ("Number of training samples: %d" %nrTrainSamples)
print ("Number of validation  samples: %d" %nrValidationSamples)
print ("Dimension of X_train: %d" %dimX)
print ("Dimension of Y_validation: %d" %dimY)

Finished: data loaded. Stats below: 
Number of training samples: 154033
Number of validation  samples: 22005
Dimension of X_train: 2351
Dimension of Y_validation: 1


In [6]:
# %%time
# with open( "../../Data/Propensity_Modeling/Neuron_Net/X_train.p", "wb" ) as f:
#     pickle.dump(X_train, f)
# with open( "../../Data/Propensity_Modeling/Neuron_Net/Y_train.p", "wb" ) as f:
#     pickle.dump(Y_train, f)
# with open( "../../Data/Propensity_Modeling/Neuron_Net/X_validataion.p", "wb" ) as f:
#     pickle.dump(X_validataion, f)
# with open( "../../Data/Propensity_Modeling/Neuron_Net/Y_validation.p", "wb" ) as f:
#     pickle.dump(Y_validation, f)
# with open( "../../Data/Propensity_Modeling/Neuron_Net/X_test.p", "wb" ) as f:
#     pickle.dump(X_test, f)
# with open( "../../Data/Propensity_Modeling/Neuron_Net/Y_test.p", "wb" ) as f:
#     pickle.dump(Y_test, f)

---
## 1. Parameter Setting
---

In [7]:
sess = tf.InteractiveSession()
BATCH_SIZE    = 100
MAX_ITERS     = 50
LEARNING_RATE = 0.1
require_improvement = 50  # Stop optimization if no improvement found in this many iterations.
n_hidden=256           # 256 units in this hidden lapyer

## 2. Model Design

### 2.1 Layers

In [8]:
# ---------------------------------------------------------------
# Linear layer:
def LinearLayer(x,dim1,dim2):
    # the W & b here are random normal (before training)
    W=tf.Variable(tf.random_normal([dim1,dim2],stddev=0.01))
    b=tf.Variable(tf.random_normal([1, dim2],stddev=0.01))
    return tf.matmul(x,W)+b

# ---------------------------------------------------------------    
# ReLU hidden layer: (one linear layer inside)
# ReLU function itself wouldn't change dimension of data.
# y = max(0, x)
def ReLULayer(x,dim1,dim2,dropout_prob=0.5):
    
    W_h = tf.Variable(tf.truncated_normal([dim1,dim2], stddev=0.1))
    b_h = tf.constant(0.1, shape=[dim2])
    z_h = tf.matmul(x,W_h) + b_h
    y_h = tf.nn.relu(z_h) 
    
    # Dropout layer
    output = tf.nn.dropout(y_h, dropout_prob)
    return output

### 2.2 Models
- **2 hidden ReLU + Sigmoid**

In [9]:
# Construct Model has 2 hidden ReLU layers.
def Model(x,dim1,dim2):
    # apply dropout to input layer
    prob_keep_input=0.8
    x=tf.nn.dropout(x,prob_keep_input)
    
    dimH1, dimH2=n_hidden, n_hidden
    
#     # hidden layer 1:
#     h1=ReLULayer(x,dim1,dimH1,dropout_prob=0.5)
#     # hidden layer 2:
#     h2=ReLULayer(h1,dimH1,dimH2,dropout_prob=0.5)
    
    # output layer (Sigmoid layer):
    # Sigmoid layer doesm't change dimension of data.
    logits=LinearLayer(x,dim1,dim2)
#     logits=LinearLayer(h2,dimH2,dim2)
    y_pred_prob=tf.nn.sigmoid(logits)
    # y_ is the predicted label
    
    return [logits, y_pred_prob]

# ---------------------------------------------------------------
# Generic training function:
# ---------------------------------------------------------------
# x                   tf.palceholder or INPUT
# y                   tf.palceholder or OUTPUT
# train_op            training operation (tf.Optimizer)
# accuracy     
# trainX, trainY      training data and labels
# testX, testY        testing data and labels

### 2.3 Model Training

In [10]:
def trainModel(accuracy, train_op, x, y, X_train, Y_train, X_validation, Y_validation, X_test, Y_test):
    # Ensure we update the global variables rather than local copies.
#     global best_validation_accuracy
#     global last_improvement

    best_validation_accuracy=0
    last_improvement=0
    
    print ("===============================")
    print ("Training started........")

    # Start-time used for printing time-usage below.
    start_time = time.time()
    
    for indexIter in range(MAX_ITERS):
        for startIndex, endIndex in zip( range(0,len(X_train),BATCH_SIZE), range(BATCH_SIZE,len(X_train),BATCH_SIZE)):
            # gradient descending, finding parameters.
            sess.run(train_op, feed_dict = {x: X_train[startIndex:endIndex], y: Y_train[startIndex:endIndex]})
            
        # Visualize accuracy only every 10 iterations
        if (indexIter%10==0) or (indexIter == (MAX_ITERS - 1)):
            acc_train = sess.run(accuracy, feed_dict={x:X_train, y:Y_train})
            acc_validation  = sess.run(accuracy, feed_dict={x:X_validation,  y:Y_validation})
            acc_test = sess.run(accuracy, feed_dict={x:X_test,  y:Y_test})
            
            if acc_validation > best_validation_accuracy :
                # Update the best-known validation accuracy.
                best_validation_accuracy = acc_validation
                
                # Set the iteration for the last improvement to current.
                last_improvement = indexIter

                # A string to be printed below, shows improvement found.
                improved_str = '*'
                print('Iteration %d: Accuracy %.5f(training) %.5f(validation) %.5f(test)'%(indexIter+1, acc_train, acc_validation, acc_test)+improved_str)
                
                # Save all variables of the TensorFlow graph to file.
                saver = tf.train.Saver()
                save_dir='../../Data/Propensity_Modeling/Neuron_Net/model01'
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                saver.save(sess, save_dir)
                print("Model saved in file: %s" % save_dir)
                
            else:
                # An empty string to be printed below.
                # Shows that no improvement was found.
                improved_str = ''
                print('Iteration %d: Accuracy %.5f(training) %.5f(validation) %.5f(test)'%(indexIter+1, acc_train, acc_validation, acc_test)+improved_str)                

        if (indexIter - last_improvement) > require_improvement:
            print("No improvement found in a while, early stop the training.")
            print ("===============================")
            break
                  
    print ("Training finished.")
    # Ending time.
    end_time = time.time()

    # Difference between start and end-times.
    time_dif = end_time - start_time

    # Print the time-usage.
    print("Time usage: " + str(datetime.timedelta(seconds=int(round(time_dif)))))
    print ("===============================")
    

### 2.4 Main function

In [11]:
# ---------------------------------------------------------------
def main():
    
    #============================
    # 1. Build model (graph)
    #============================
    
    x = tf.placeholder(tf.float32, shape=[None, dimX]) 
    # None is the number of data set, 784 is the dimensions of features
    y = tf.placeholder(tf.float32, shape=[None, dimY])
    
    # model returns a score vector over all classes
    [logits,y_pred_prob] = Model(x, dimX, dimY)
    
    # Get predicted classes
    y_pred = tf.round(y_pred_prob)
    
    # Loss function (Cross-entropy for classification)
    cross_entropy=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits = logits))
    # Error: Only call `sigmoid_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)
    
    # Accuracy(if different measure from loss)
    correct_prediction = tf.equal(y_pred, y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    #============================
    # 2. Train model
    #============================
    # initialize variables
    tf.global_variables_initializer().run()
    
    # Best validation accuracy seen so far.
    best_validation_accuracy= 0
    # Iteration-number for last improvement to validation accuracy.
    last_improvement = 0

    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()

    # Train model:
    train_op = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
    trainModel(accuracy, train_op, x, y, X_train, Y_train, X_validation, Y_validation, X_test, Y_test)

    acc_test = sess.run(accuracy, feed_dict={x:X_test, y:Y_test})
    y_pred=sess.run(y_pred, feed_dict={x:X_test})
    y_pred_prob=sess.run(y_pred_prob, feed_dict={x:X_test})

    print('Final model (may note be the optimal) on test data got accuracy:', acc_test)
    
    return [y_pred_prob, y_pred]

---
## 3. Trained Result:
---
### 3.1 Training Result & Accuracy

In [12]:
#(REPRODUCIBILITY) set random seeds
tf.set_random_seed(722)

[y_pred_prob, y_pred]=main()

# y_true = np.argmax(mnist.test.labels,1)
    
# Save the the whole session
# saver = tf.train.Saver()
# name_of_checkpoint = "model_C"
# print("\n Saving model to: " + name_of_checkpoint)
# saver.save(sess,name_of_checkpoint)

Training started........
Iteration 1: Accuracy 0.84240(training) 0.85267(validation) 0.83722(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Iteration 11: Accuracy 0.84210(training) 0.85249(validation) 0.83756(test)
Iteration 21: Accuracy 0.84196(training) 0.85231(validation) 0.83744(test)
Iteration 31: Accuracy 0.84230(training) 0.85253(validation) 0.83749(test)
Iteration 41: Accuracy 0.15895(training) 0.14833(validation) 0.16340(test)
Iteration 50: Accuracy 0.84211(training) 0.85290(validation) 0.83737(test)*
Model saved in file: ../../Data/Propensity_Modeling/Neuron_Net/model01
Training finished.
Time usage: 0:07:56
Final model (may note be the optimal) on test data got accuracy: 0.837397


In [14]:
print(y_pred.shape)
y_pred

(44009, 1)


array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 0.],
       [ 0.],
       [ 0.]], dtype=float32)

In [15]:
y_pred_prob

array([[ 0.],
       [ 0.],
       [ 0.],
       ..., 
       [ 0.],
       [ 0.],
       [ 0.]], dtype=float32)

In [13]:
sum(Y_train==0)/len(Y_train)

array([ 0.84684451])

### 3.2 Confusion Matrix

In [ ]:
# Compute confusion matrix
y_Test = tf.argmax(testY, 1)
y_test=y_Test.eval()
cfm = confusion_matrix(y_test, y_pred,labels=range(0,10))
np.set_printoptions(precision=2)

print('\nPlot of Confusion Matrix without nomalize is:')
plt.figure(figsize=(14,10))
plot_confusion_matrix(cfm, classes=range(0,10), normalize=False)
plt.show()
print('\nPlot of Confusion Matrix with nomalize is:')
plt.figure(figsize=(14,10))
plot_confusion_matrix(cfm, classes=range(0,10), normalize=True)
plt.show()

### End
---